<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Indexing" data-toc-modified-id="Indexing-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Indexing</a></span><ul class="toc-item"><li><span><a href="#loc:-label-based-indexing" data-toc-modified-id="loc:-label-based-indexing-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>loc: label-based indexing</a></span></li><li><span><a href="#iloc:-integer-positional-indexing" data-toc-modified-id="iloc:-integer-positional-indexing-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>iloc: integer positional indexing</a></span></li><li><span><a href="#boolean-mask-indexing" data-toc-modified-id="boolean-mask-indexing-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>boolean mask indexing</a></span></li><li><span><a href="#query()-method" data-toc-modified-id="query()-method-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>query() method</a></span></li></ul></li><li><span><a href="#Grouping" data-toc-modified-id="Grouping-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Grouping</a></span></li><li><span><a href="#Reshaping" data-toc-modified-id="Reshaping-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Reshaping</a></span><ul class="toc-item"><li><span><a href="#wide-to-long-(tidyr::gather)" data-toc-modified-id="wide-to-long-(tidyr::gather)-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>wide-to-long (tidyr::gather)</a></span></li><li><span><a href="#long-to-wide-(tidyr::spread)" data-toc-modified-id="long-to-wide-(tidyr::spread)-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>long-to-wide (tidyr::spread)</a></span></li></ul></li><li><span><a href="#Merge-&amp;-Joins" data-toc-modified-id="Merge-&amp;-Joins-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Merge &amp; Joins</a></span><ul class="toc-item"><li><span><a href="#join" data-toc-modified-id="join-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>join</a></span></li><li><span><a href="#merge" data-toc-modified-id="merge-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>merge</a></span></li><li><span><a href="#semi-join-with-isin" data-toc-modified-id="semi-join-with-isin-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>semi-join with isin</a></span></li></ul></li></ul></div>

In [1]:
import pandas as pd
import numpy as np

## Indexing

In [2]:
np.random.seed(0)
df1 = pd.DataFrame(np.random.randn(6,4), index=list('abcdef'), columns=list('ABCD'))
df1

,A,B,C,D
a,1.764052,0.400157,0.978738,2.240893
b,1.867558,-0.977278,0.950088,-0.151357
c,-0.103219,0.410599,0.144044,1.454274
d,0.761038,0.121675,0.443863,0.333674
e,1.494079,-0.205158,0.313068,-0.854096
f,-2.552990,0.653619,0.864436,-0.742165


### loc: label-based indexing

In [3]:
# get columns 'A' & 'C' of all rows with index labels 'a' or 'c' 
df1.loc[['a', 'c'], ['A', 'C']]

# returns Series
df1.loc['a', ['A', 'C']]

A    1.764052
C    0.978738
Name: a, dtype: float64

### iloc: integer positional indexing

In [4]:
# Get 1st, 2nd, and 4th row and 1st and 2nd column: 
df1.iloc[[0, 1, 3], [0, 1]]

# Get all rows and columns 2-9 if present (range is exclusive)
df1.iloc[:, 2:10]

# Get 2nd and 3rd row
df1.iloc[1:3]

,A,B,C,D
b,1.867558,-0.977278,0.950088,-0.151357
c,-0.103219,0.410599,0.144044,1.454274


### boolean mask indexing

In [5]:
df1[(df1.A < 0) & (df1['C'] > 0)]

# SQL: select C from df where A < 1
df1.loc[df1.A < 1, 'C']

c    0.144044
d    0.443863
f    0.864436
Name: C, dtype: float64

### query() method
Nice alternative for boolean indexing, even works with indexes

In [6]:
# default index name is 'index'
df1.query("A < 1 & 'a' < index <= 'd'")

,A,B,C,D
c,-0.103219,0.410599,0.144044,1.454274
d,0.761038,0.121675,0.443863,0.333674


## Grouping

In [7]:
df2 = pd.DataFrame({'A' : ['foo', 'foo', 'foo', 'bar', 'bar'],
                    'B' : ['one', 'one', 'two', 'two', 'two'],
                    'C' : [1, 2, 3, 4, 5],
                    'D' : [1, 1, 1, 1, 1]})

In [8]:
# Series with MultiIndex (A,B) and values=max(C) per A
s = df2.groupby(['A','B'])['C'].sum()
s.to_frame('sum_C')

sum_C
A   B         
bar two      9
foo one      3
    two      3

In [9]:
df2.groupby(['A','B']).agg({'C': sum, 'D': lambda x: len(x)})

C  D
A   B        
bar two  9  2
foo one  3  2
    two  3  1

In [10]:
# Series with index=order_id (like index before) and values=max(order_number)
# Data has the same number as rows as before, i.e. groupby('user_id') does not return one row per user_id!
df2['E'] = df2.groupby('A')['C'].transform(np.mean)
df2

,A,B,C,D,E
0,foo,one,1,1,2.0
1,foo,one,2,1,2.0
2,foo,two,3,1,2.0
3,bar,two,4,1,4.5
4,bar,two,5,1,4.5


## Reshaping

In [11]:
# df1 is in wide format, with A-D as metric columns and the index to identify a row
df1.head(3)

,A,B,C,D
a,1.764052,0.400157,0.978738,2.240893
b,1.867558,-0.977278,0.950088,-0.151357
c,-0.103219,0.410599,0.144044,1.454274


### wide-to-long (tidyr::gather)

In [12]:
df1_long = df1.reset_index().melt(id_vars='index', var_name='kpi', value_name='value') #  deletes the existing index
df1_long
# pd.wide_to_long() can also be used, useful when metric columns share the same prefix

,index,kpi,value
0,a,A,1.764052
1,b,A,1.867558
2,c,A,-0.103219
3,d,A,0.761038
4,e,A,1.494079
5,f,A,-2.552990
6,a,B,0.400157
7,b,B,-0.977278
8,c,B,0.410599
9,d,B,0.121675


stack() is another option which "stacks" the columns into a new index level

In [13]:
# to create a dataframe like melt() does:
df1.stack().to_frame('value').reset_index(level=1).rename(columns={'level_1': 'kpi'})
df1_stacked = df1.stack()
df1_stacked

a  A    1.764052
   B    0.400157
   C    0.978738
   D    2.240893
b  A    1.867558
   B   -0.977278
   C    0.950088
   D   -0.151357
c  A   -0.103219
   B    0.410599
   C    0.144044
   D    1.454274
d  A    0.761038
   B    0.121675
   C    0.443863
   D    0.333674
e  A    1.494079
   B   -0.205158
   C    0.313068
   D   -0.854096
f  A   -2.552990
   B    0.653619
   C    0.864436
   D   -0.742165
dtype: float64

### long-to-wide (tidyr::spread)

In [14]:
df1_long.pivot(index='index', columns='kpi', values='value')

kpi,A,B,C,D
index,,,,
a,1.764052,0.400157,0.978738,2.240893
b,1.867558,-0.977278,0.950088,-0.151357
c,-0.103219,0.410599,0.144044,1.454274
d,0.761038,0.121675,0.443863,0.333674
e,1.494079,-0.205158,0.313068,-0.854096
f,-2.552990,0.653619,0.864436,-0.742165


In [15]:
df1_stacked.unstack()

,A,B,C,D
a,1.764052,0.400157,0.978738,2.240893
b,1.867558,-0.977278,0.950088,-0.151357
c,-0.103219,0.410599,0.144044,1.454274
d,0.761038,0.121675,0.443863,0.333674
e,1.494079,-0.205158,0.313068,-0.854096
f,-2.552990,0.653619,0.864436,-0.742165


## Merge & Joins

In [16]:
left  = pd.DataFrame({'A': ['A0', 'A1', 'A2'], 'B': ['B0', 'B1', 'B2']}, index=['K0', 'K1', 'K2']).rename_axis('l_index')
right = pd.DataFrame({'C': ['C0', 'C2', 'C3'], 'D': ['D0', 'D2', 'D3']}, index=['K0', 'K2', 'K3']).rename_axis('r_index')

### join
DataFrame.join defaults to a LEFT JOIN and is best suited for joining DataFrames on their indexes or joining a DataFrame with a Series.

In [17]:
# left join on indexes
left.join(right)

# inner join on indexes
left.join(right, how='inner')

# for left DF a column name to join on can be specified
# right DF is always joined using index
left.reset_index().join(right, on='l_index')

# join also works with a Series that has its name field set
# same as: left['C'] = right['C']
left.join(right['C'])

,A,B,C
l_index,,,
K0,A0,B0,C0
K1,A1,B1,NaN
K2,A2,B2,C2


### merge
DataFrame.merge defaults to an INNER JOIN and is best suited for joining on columns.

Note: when joining on columns the indexes are not preserved!

In [18]:
# inner join based on intersection of columns
left.merge(left)

# inner join on indexes using merge
left.merge(right, left_index=True, right_index=True)

,A,B,C,D
K0,A0,B0,C0,D0
K2,A2,B2,C2,D2


### semi-join with isin
In the official documentation, Series.isin is is listed in the Indexing section since it returns a boolean mask.

However, it can also be used for semi joins.

In [19]:
# SELECT * FROM left WHERE A IN (SELECT C FROM right)
left[left.A.isin(right.C)]

# semi-join using indexes
left[left.index.isin(right.index)]

# semi-join using indexes with .loc
left.loc[left.index.isin(right.index), 'B']

l_index
K0    B0
K2    B2
Name: B, dtype: object